In [2]:
import nltk
import pandas as pd
import numpy as np
import os

#### Import Data

In [19]:
DATA_DIR = "processed data"
FILE = "english_only_question_and_answer_without_instructor.csv"
dat = pd.read_csv(os.getcwd() + '/' + DATA_DIR + '/' + FILE, encoding = 'ISO-8859-1')
# dat
posts = dat[dat['post_content_length']>3]['post_content']
# posts

In [20]:
NO_DOCUMENT = len(posts)
NO_DOCUMENT

931

In [21]:
corpus = posts

#### Prepocessing

In [23]:
from nltk import word_tokenize
from nltk.corpus import stopwords
en_stop = set(stopwords.words('english'))
en_stop.add("'s")
en_stop.add("'re")
en_stop.add("'m")
en_stop.add("'ve")
en_stop.add("n't")
en_stop.add("would")
en_stop.add("really")
en_stop.add("may")
en_stop.add("even")
en_stop.add("also")
en_stop.add("us")
import string
exclude = set(string.punctuation)
exclude.add("--")
exclude.add("\'\'")
exclude.add("...")
exclude.add("``")
exclude.add("..")
from nltk.stem import WordNetLemmatizer

In [24]:
def convert_tag(tag):
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return 'n'

In [25]:
def clean(doc):
    tokenized_lower = word_tokenize(doc.lower()) 
    tokenized_lower = [word for word in tokenized_lower if len(word)>2]
    stop_free = [word for word in tokenized_lower if word not in en_stop]
    punc_free = [d for d in stop_free if d not in exclude]
    tags = nltk.pos_tag(punc_free)
    tags = [convert_tag(tag[1]) for tag in tags]
    lemmatized = [WordNetLemmatizer().lemmatize(word, tag) for word,tag in zip(punc_free,tags)]
    return lemmatized

In [26]:
normalized_corpus = [clean(d) for d in corpus]
len(normalized_corpus)

931

#### Create dictionary

In [27]:
import gensim
from gensim import corpora

In [28]:
# id2word.dfs[id2word.token2id['.']]

In [29]:
# id2word.token2id['hi']

In [30]:
# create dictionary
id2word = corpora.Dictionary(normalized_corpus)
len(id2word)

5116

In [31]:
# create filtered bog representation
id2word.filter_extremes(no_below=3, no_above=0.1)
bog_corpus_filtered = [id2word.doc2bow(doc) for doc in normalized_corpus]
len(id2word)

1401

In [32]:
doc_wids = [id2word.doc2idx(doc) for doc in normalized_corpus]
print(doc_wids[0])

[4, 21, -1, 9, -1, -1, -1, -1, 12, -1, -1, 32, -1, 28, 34, 31, 35, 5, 33, 35, 31, -1, 35, 31, -1, 20, 18, 11, 8, -1, 16, 24, -1, 2, 31, -1, 31, 30, -1, 22, 24, 2, 38, 6, 9, 35, 24, 7, -1, -1, 31, 29, 14, -1, 31, -1, 19, -1, 10, 27, 31, 1, 0, 23, 13, 32, 35, -1, 26, 15, 17, -1, 37, 25, 3, 36]


In [33]:
doc_wids = [[str(i) for i in doc if i!=-1] for doc in doc_wids]
print(doc_wids[0])

['4', '21', '9', '12', '32', '28', '34', '31', '35', '5', '33', '35', '31', '35', '31', '20', '18', '11', '8', '16', '24', '2', '31', '31', '30', '22', '24', '2', '38', '6', '9', '35', '24', '7', '31', '29', '14', '31', '19', '10', '27', '31', '1', '0', '23', '13', '32', '35', '26', '15', '17', '37', '25', '3', '36']


In [34]:
doc_wids = [" ".join(doc) for doc in doc_wids]
doc_wids[0]

'4 21 9 12 32 28 34 31 35 5 33 35 31 35 31 20 18 11 8 16 24 2 31 31 30 22 24 2 38 6 9 35 24 7 31 29 14 31 19 10 27 31 1 0 23 13 32 35 26 15 17 37 25 3 36'

In [35]:
# doc_wids

In [36]:
# output doc_wids
fileObject = open(os.pardir + '/BTM-master/arvr/arvr_wids.txt', 'w')
for doc in doc_wids:
    fileObject.write(doc)
    fileObject.write('\n')
fileObject.close()

In [37]:
# output dictionary
fileObject = open(os.pardir + '/BTM-master/arvr/arvr_dict.txt', 'w')
for i in id2word.items():
    fileObject.write(str(i[0])+'\t'+i[1])
    fileObject.write('\n')
fileObject.close()